# Setup

## Imports

In [25]:
# Setup: imports, settings, and helpers
import os
import sys
import json
import math
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from scipy.stats import fisher_exact

import warnings

## Notebook Configuration

In [108]:
warnings.filterwarnings('ignore')
np.set_printoptions(precision=4, suppress=True)
pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 160)

# Paths
DATA_PATH = 'lending_club_dataset.csv'
SCHEMA_VALIDATION_PATH = 'schema_validation_sheet.csv'

## Loading the Dataset and Schema Validation Sheet

In [3]:
# Utility: safe read CSV with basic dtype hints
def read_csv_safely(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Dataset not found at {path}. Please ensure the CSV is in the workspace root.")
    # Low-memory to reduce dtype inference churn
    return pd.read_csv(path, low_memory=False)

In [4]:
# Load dataset
try:
    df_raw = read_csv_safely(DATA_PATH)
    print("Loaded dataset with shape:", df_raw.shape)
except Exception as e:
    print("Failed to load dataset:", e)
    raise

Loaded dataset with shape: (10000, 28)


In [109]:
# Load dataset
try:
    df_schema_validation = read_csv_safely(SCHEMA_VALIDATION_PATH)
    print("Loaded dataset with shape:", df_schema_validation.shape)
except Exception as e:
    print("Failed to load dataset:", e)
    raise

Loaded dataset with shape: (28, 10)


## Constants

In [44]:
SEED = 42 # Random seed for reproducibility

# Split Ratios
TRAIN_SIZE = 0.8  # Proportion of data for training set
VAL_SIZE = 0.1  # Proportion of data for validation set
TEST_SIZE = 0.1  # Proportion of data for test set

# Columns
TARGET_COL = 'is_bad'  # Target column for classification
NUMERICAL_COLS = ["annual_inc","delinq_2yrs"]
CAT_COLS = ["home_ownership", "verification_status", "pymnt_plan", "purpose_cat", "zip_code", "addr_state", ]
# CONT_COLS = ["annual_inc", "debt_to_income", ""]
# TEXT_COLS = ["Notes", "purpose", ]
# RAT_COLS = ["delinq_2yrs"]
# DISCRETE_COLS = ["open_acc", "pub_rec", "total_acc", "acc_now_delinq", "tot_coll_amt", "tot_cur_bal", "total_rev_hi_lim"]
# DATE_COLS = ["earliest_cr_line"]

In [6]:
# Check that MISSING_VALUE_FILL is not in the dataset
if MISSING_VALUE_FILL in df_raw.values:
    raise ValueError(f"MISSING_VALUE_FILL value {MISSING_VALUE_FILL} already exists in the dataset. Choose a different placeholder.")

In [45]:
drop_cols = []
splits = []

# Functions

In [ ]:
# column description
def describe_column(df: pd.DataFrame, col: str):
    print(f"Missing Values: {df[col].isnull().sum()}")
    print(f"Unique Values: {df[col].nunique()}")

    try:
        print(f"Min Value: {df[col].min()}")
        print(f"Max Value: {df[col].max()}")
    except:
        pass

    print(df[col].value_counts(dropna=False).head(10))
    print()
    print(df[col].describe(include='all'))

In [115]:
def check_against_schema(df: pd.DataFrame, schema_df: pd.DataFrame):
    issues = {}

    cols_missing_from_schema = set(df.columns) - set(schema_df.columns)
    issues['missing_from_schema'] = list(cols_missing_from_schema)
    cols_missing_from_dataframe = set(schema_df.columns) - set(df.columns)
    issues['missing_from_dataframe'] = list(cols_missing_from_dataframe)

    for _, row in schema_df.iterrows():
        col = row['column']
        expected_dtype = row['expected_dtype']
        unique_min = row['unique_min']
        unique_max = row['unique_max']
        value_min = row['value_min']
        value_max = row['value_max']
        
        if col in df.columns:            
            col_issues = []
            if expected_dtype:
                actual_dtype = str(df[col].dtype)
                if expected_dtype not in actual_dtype:
                    col_issues.append(f"Expected dtype {expected_dtype}, found {actual_dtype}.")
            if pd.notnull(unique_min):
                unique_count = df[col].nunique()
                if unique_count < unique_min:
                    col_issues.append(f"Unique count {unique_count} less than minimum {unique_min}.")
            if pd.notnull(unique_max):
                unique_count = df[col].nunique()
                if unique_count > unique_max:
                    col_issues.append(f"Unique count {unique_count} greater than maximum {unique_max}.")
            if pd.notnull(value_min):
                try:
                    min_value = df[col].min()
                    if min_value < value_min:
                        col_issues.append(f"Min value {min_value} less than minimum {value_min}.")
                except:
                    pass
            if pd.notnull(value_max):
                try:
                    max_value = df[col].max()
                    if max_value > value_max:
                        col_issues.append(f"Max value {max_value} greater than maximum {value_max}.")
                except:
                    pass
            if col_issues:
                issues[col] = col_issues
        
        actual_dtype = str(df[col].dtype)
        if expected_dtype not in actual_dtype:
            issues[col] = f"Column {col} has dtype {actual_dtype}, expected {expected_dtype}."

    return issues

check_against_schema(df_raw, df_schema_validation)

KeyError: 'expected_dtype'

## Statistical Tests

### Fisher's Exact Test

In [76]:
def fisher_exact_test(df: pd.DataFrame, feature_col: str, positive_feature: int, target_col: str):
    contingency_table = pd.crosstab(df[feature_col] == positive_feature, df[target_col])

    print("Contingency table:\n", contingency_table)

    # Fisher's exact test
    odds_ratio, p_value = fisher_exact(contingency_table)

    print(f"Fisher's Exact Test odds ratio: {odds_ratio:.4f}")
    print(f"p-value: {p_value:.4g}")

# Pre-split EDA

In [15]:
def summarize_near_constant_features(
    df: pd.DataFrame,
    *,
    dominant_thresh: float = 0.98,   # flag if top value covers ≥ this fraction of non-null values
    max_unique_for_flag: int | None = None,  # optionally also require unique values ≤ this number
    min_non_null: int = 5,           # skip columns with too few non-nulls
    treat_bool_as_categorical: bool = True
) -> pd.DataFrame:
    """
    Analyze columns for 'near-constant' behavior and return a summary DataFrame.

    Columns are flagged when the dominant (most frequent) value's share among non-null values
    is ≥ dominant_thresh. Optionally (if max_unique_for_flag is set), we also require that the
    number of unique non-null values ≤ max_unique_for_flag.

    Parameters
    ----------
    df : pd.DataFrame
        Input data.
    dominant_thresh : float, default 0.98
        Threshold for dominant value proportion to flag near-constant.
    max_unique_for_flag : int or None, default None
        If set, near-constant flag additionally requires unique_count ≤ this.
        (E.g., set to 2 to flag only nearly-all-0/1 columns.)
    min_non_null : int, default 5
        Skip columns with fewer than this many non-null observations.
    treat_bool_as_categorical : bool, default True
        If True, boolean columns are summarized as categoricals.

    Returns
    -------
    pd.DataFrame
        Columns:
        - column
        - dtype
        - non_null_count
        - missing_rate
        - unique_count
        - dominant_value
        - dominant_count
        - dominant_share
        - minority_count
        - variance (numeric only; else NaN)
        - std (numeric only; else NaN)
        - min_value (numeric only; else NaN)
        - max_value (numeric only; else NaN)
        - entropy_bits (Shannon entropy base-2 on value distribution)
        - is_binary_like (unique_count == 2)
        - near_constant_flag (boolean)
        - top_5_values (string formatted "value:count; …")
    """
    summaries = []

    for col in df.columns:
        s = df[col]
        non_null = s.dropna()
        non_null_count = int(non_null.shape[0])
        missing_rate = (1.0 - (non_null_count / max(1, s.shape[0])))*100

        # Handle dtype classification early so branches can use it
        is_bool = pd.api.types.is_bool_dtype(s)
        is_numeric = pd.api.types.is_numeric_dtype(s) and not (is_bool and treat_bool_as_categorical)

        # Precompute min/max for numeric columns
        if is_numeric and non_null_count > 0:
            try:
                min_value = float(non_null.astype(float).min())
                max_value = float(non_null.astype(float).max())
            except Exception:
                # Fallback in case of mixed numeric types that fail astype(float)
                min_value = float(pd.to_numeric(non_null, errors='coerce').min())
                max_value = float(pd.to_numeric(non_null, errors='coerce').max())
        else:
            min_value = np.nan
            max_value = np.nan

        # # Top-5 most common values (include NaN label consistent with helper)
        # try:
        #     top5_vals = top5_summary(s)
        # except Exception:
        #     # If helper not available for any reason, compute a simple fallback
        vc_tmp = s.value_counts(dropna=False).head(5)
        def _fmt(v):
            if pd.isna(v):
                return 'NaN'
            txt = str(v)
            return (txt[:60] + '…') if len(txt) > 60 else txt
        top5_vals = "; ".join([f"{_fmt(idx)}:{int(cnt)}" for idx, cnt in vc_tmp.items()])

        if non_null_count < min_non_null:
            # Not enough data to assess; still record info
            summaries.append({
                "column": col,
                "dtype": s.dtype.name,
                "non_null_count": non_null_count,
                "missing_rate": missing_rate,
                "unique_count": non_null.nunique(dropna=True),
                "min_value": min_value,
                "max_value": max_value,
                "dominant_value": np.nan,
                "dominant_count": 0,
                "dominant_share": np.nan,
                "minority_count": 0,
                "variance": np.nan,
                "std": np.nan,
                "entropy_bits": np.nan,
                "is_binary_like": False,
                "near_constant_flag": False,
                "top_5_values": top5_vals,
            })
            continue

        # Value counts for frequencies
        vc = non_null.value_counts(dropna=False)
        dominant_value = vc.index[0]
        dominant_count = int(vc.iloc[0])
        unique_count = int(vc.shape[0])
        dominant_share = dominant_count / non_null_count
        minority_count = non_null_count - dominant_count

        # Entropy (base-2)
        probs = (vc / non_null_count).to_numpy()
        entropy_bits = float(-np.sum(probs * np.log2(probs))) if unique_count > 1 else 0.0

        # Numeric variance/std if applicable
        if is_numeric:
            variance = float(non_null.astype(float).var(ddof=1)) if non_null_count > 1 else 0.0
            std = float(np.sqrt(variance))
        else:
            variance = np.nan
            std = np.nan

        # Binary-like flag
        is_binary_like = (unique_count == 2)

        # Near-constant logic
        meets_share = dominant_share >= dominant_thresh
        meets_unique = True if max_unique_for_flag is None else (unique_count <= max_unique_for_flag)
        near_constant_flag = bool(meets_share and meets_unique)

        summaries.append({
            "column": col,
            "dtype": s.dtype.name,
            "non_null_count": non_null_count,
            "missing_rate": missing_rate,
            "unique_count": unique_count,
            "min_value": min_value,
            "max_value": max_value,
            "dominant_value": dominant_value,
            "dominant_count": dominant_count,
            "dominant_share": dominant_share,
            "minority_count": minority_count,
            "variance": variance,
            "std": std,
            "entropy_bits": entropy_bits,
            "is_binary_like": is_binary_like,
            "near_constant_flag": near_constant_flag,
            "top_5_values": top5_vals,
        })

    out = pd.DataFrame(summaries)
    # Order: flags first, then by dominant_share desc, then low entropy
    # out = out.sort_values(
    #     by=["near_constant_flag", "dominant_share", "entropy_bits"],
    #     ascending=[False, False, True],
    #     kind="mergesort"
    # ).reset_index(drop=True)
    return out

# Recompute summary with new columns
df_summary = summarize_near_constant_features(df_raw)

In [107]:
df_summary[3:4]

,column,dtype,non_null_count,missing_rate,unique_count,min_value,max_value,dominant_value,dominant_count,dominant_share,minority_count,variance,std,entropy_bits,is_binary_like,near_constant_flag,top_5_values
3,emp_length,object,10000,0.0,14,NaN,NaN,10,2160,0.216,7840,NaN,NaN,3.137702,False,False,10:2160; 1:2083; 2:1183; 3:1010; 4:889


In [58]:
drop_cols.append("pymnt_plan")

# Pre-split Transformations

In [26]:
df_presplit_trans = df_raw.copy()

In [28]:
encoder = LabelEncoder()
df_presplit_trans["initial_list_status"] = encoder.fit_transform(df_raw["initial_list_status"])

## collections_12_mths_ex_med
Type of missingness is unknown at this time.

In [ ]:
# # collections_12_mths_ex_med consists of 0s and missing values; will convert missing values to -1 and examine further after post-split
# print(df_raw["collections_12_mths_ex_med"].value_counts())
# print(df_raw["collections_12_mths_ex_med"].isna().sum())

# #convert "NA" values to -1
# df_raw["collections_12_mths_ex_med"] = df_raw["collections_12_mths_ex_med"].fillna(-1)
# print()
# print(df_raw["collections_12_mths_ex_med"].value_counts())
# print(df_raw["collections_12_mths_ex_med"].isna().sum())

# Splitting the dataset

In [46]:
target_col = 'is_bad'
X = df_presplit_trans.drop(columns=[target_col])
y = df_presplit_trans[target_col]

# First: split off the test set (10%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y,
    test_size=0.10,
    stratify=y,
    random_state=42
)

# Next: split the remaining 90% into train (80%) and validation (10%)
# Since test already took 10%, we want 10/90 = ~0.1111 of the remainder as validation
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=1/9,   # ≈ 0.1111
    stratify=y_temp,
    random_state=42
)

splits = [X_train, X_val, X_test, y_train, y_val, y_test]    

print("Train size:", len(X_train))
print("Validation size:", len(X_val))
print("Test size:", len(X_test))

# Check stratification worked (class balance preserved)
print("\nClass distribution in train:")
print(y_train.value_counts(normalize=True))

print("\nClass distribution in validation:")
print(y_val.value_counts(normalize=True))

print("\nClass distribution in test:")
print(y_test.value_counts(normalize=True))

Train size: 8000
Validation size: 1000
Test size: 1000

Class distribution in train:
is_bad
0    0.8705
1    0.1295
Name: proportion, dtype: float64

Class distribution in validation:
is_bad
0    0.871
1    0.129
Name: proportion, dtype: float64

Class distribution in test:
is_bad
0    0.87
1    0.13
Name: proportion, dtype: float64


# Post-split EDA

In [35]:
df_analysis = df_presplit_trans[df_presplit_trans['Id'].isin(X_train['Id'])]

## collections_12_mths_ex_med

In [39]:
# collections_12_mths_ex_med has 29 missing values and by itself likely contains little information for prediction; confirming with Fisher's exact test
df_analysis["collections_12_mths_ex_med"] = df_analysis["collections_12_mths_ex_med"].fillna(-1)
df_analysis["collections_12_mths_ex_med"].value_counts()
fisher_exact_test(df_analysis, 'collections_12_mths_ex_med', -1, TARGET_COL)

Contingency table:
 is_bad                         0     1
collections_12_mths_ex_med            
False                       6936  1035
True                          28     1
Fisher's Exact Test odds ratio: 0.2393
p-value: 0.1669


In [57]:
drop_cols.append('collections_12_mths_ex_med')

## initial_list_status

In [42]:
# initial_list_status
col = 'initial_list_status'
print(f"Missing values in {col}: {df_analysis[col].isna().sum()}")
print(df_analysis[col].value_counts())

fisher_exact_test(df_analysis, col, 1, TARGET_COL)

Missing values in initial_list_status: 0
initial_list_status
0    7986
1      14
Name: count, dtype: int64
Contingency table:
 is_bad                  0     1
initial_list_status            
False                6950  1036
True                   14     0
Fisher's Exact Test odds ratio: 0.0000
p-value: 0.2399


In [63]:
drop_cols.append('initial_list_status')

# Post-Split Transformations

In [64]:
drop_cols

['collections_12_mths_ex_med',
 'collections_12_mths_ex_med',
 'pymnt_plan',
 'initial_list_status']

In [67]:
# Add KeyError exception handling
for col in drop_cols:
    if col not in X_train.columns:
        print(f"Warning: Column '{col}' not found in X_train")
    if col not in X_val.columns:
        print(f"Warning: Column '{col}' not found in X_val")
    if col not in X_test.columns:
        print(f"Warning: Column '{col}' not found in X_test")
    try:
        X_train.drop(columns=[col], inplace=True)
        X_val.drop(columns=[col], inplace=True)
        X_test.drop(columns=[col], inplace=True)
    except KeyError as e:
        print(f"KeyError: {e} - One or more columns not found in DataFrame")

KeyError: "['collections_12_mths_ex_med'] not found in axis" - One or more columns not found in DataFrame
KeyError: "['collections_12_mths_ex_med'] not found in axis" - One or more columns not found in DataFrame


In [69]:
X_train.head(5)

,Id,emp_title,emp_length,home_ownership,annual_inc,verification_status,Notes,purpose_cat,purpose,zip_code,addr_state,debt_to_income,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,mths_since_last_major_derog,policy_code
4546,4547,U.S. Patent and Trademark Office,2,RENT,49861.0,VERIFIED - income,Borrower added on 05/05/10 > My goal is to be ...,debt consolidation,Financial freedom,207xx,MD,16.00,0.0,8/1/99,2.0,NaN,NaN,5.0,0.0,13236,71.2,12.0,1,PC4
4772,4773,Hopkins Distribution co.,1,RENT,24996.0,not verified,I wish to get a loan to pay off 2 Credet cards...,debt consolidation,Debt consolidation,895xx,NV,11.38,0.0,8/1/03,0.0,NaN,NaN,10.0,0.0,6291,24.0,16.0,3,PC2
5650,5651,Moss Adams LLP,5,RENT,95000.0,VERIFIED - income source,Borrower added on 03/07/11 > I plan on getti...,debt consolidation,Loan,900xx,CA,7.41,0.0,1/1/96,0.0,61.0,NaN,7.0,0.0,10954,35.1,17.0,2,PC1
3787,3788,The Aspen Group,4,RENT,23000.0,not verified,Borrower added on 07/19/11 > I am relocating f...,moving,Moving,137xx,NY,21.97,0.0,6/1/03,2.0,NaN,NaN,9.0,0.0,1614,14.8,20.0,3,PC4
7562,7563,IFC,2,OWN,182496.0,VERIFIED - income source,"Hi all, this is a re-list - still ironing out ...",major purchase,Vineyard Property in NZ - Perfect Timing,117xx,NY,4.73,0.0,3/1/92,2.0,NaN,NaN,5.0,0.0,0,0.0,30.0,3,PC2


# Consider dropping rows with missing data

# Imputing missing data